In [1]:
import pandas as pd; import numpy as np
df = pd.concat([pd.read_csv('Whole_dataset1.txt'),pd.read_csv('Whole_dataset2.txt')\
,pd.read_csv('Whole_dataset3.txt'),pd.read_csv('Whole_dataset4.txt')])
df = df.sort_values(by='CustomerID')
cust_id = np.array(df.CustomerID)
flag = cust_id[0]; count=0
for i in range(cust_id.shape[0]):
    if(flag==cust_id[i]):
        cust_id[i]=count
    else:
        flag = cust_id[i]
        count+=1
        cust_id[i]=count
df['CustomerID'] = cust_id; del cust_id
df = df.sort_values(by='MovieID')
mov_id = np.array(df.MovieID)
flag = mov_id[0]; count=0
for i in range(mov_id.shape[0]):
    if(flag==mov_id[i]):
        mov_id[i]=count
    else:
        flag = mov_id[i]
        count+=1
        mov_id[i]=count
df['MovieID'] = mov_id; del mov_id
l = int(0.7*df.shape[0])
df = df.sort_values(by='Date')
df = df[l:]
mov_id = np.array(df.MovieID)
cust_id = np.array(df.CustomerID)
score = np.array(df.Score)
ts_data = {}
for i in range(score.shape[0]):
    ts_data[(cust_id[i],mov_id[i])] = score[i]

In [2]:
del df, mov_id, cust_id, score

In [3]:
import gc; gc.collect()

7

In [4]:
import pickle
with open('Whole-data-sparse-matrix.pkl','rb') as fp:
    SPM = pickle.load(fp)
fp.close()

In [5]:
from sklearn.decomposition import NMF
model = NMF(n_components = 100, init='random')
U = model.fit_transform(SPM)
M = model.components_
U.shape, M.shape

((480189, 100), (100, 17770))

In [6]:
import math
import numpy as np
e = 0
for i in ts_data.keys():
    row,col = i
    R = ts_data[(row,col)]
    R_h = np.dot(U[row,:],M[:,col])
    #print (R,R_h)
    e = e + math.pow((R-R_h),2)
math.sqrt(e/len(ts_data.keys()))

3.6479461566957188